In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = load_files(r"D:\Master Thesis\txt_sentoken")
movie_reviews, y = movie_data.data, movie_data.target

In [3]:
documents = []

for sen in range(0, len(movie_reviews)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(movie_reviews[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    document = document.split()

    # normalize whitespace
    document = ' '.join(document)
    
    #documents.append(document)
    documents.append(str(movie_reviews[sen]))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [8]:
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[182  26]
 [ 32 160]]
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       208
           1       0.86      0.83      0.85       192

    accuracy                           0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400

0.855


In [10]:
import eli5
eli5.show_weights(classifier, top=10)

C:\Users\Marc\Anaconda3\envs\eli5 lime\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Marc\Anaconda3\envs\eli5 lime\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Weight,Feature
0.0189 ± 0.0492,x104
0.0130 ± 0.0385,x1482
0.0076 ± 0.0256,x142
0.0072 ± 0.0228,x1287
0.0065 ± 0.0210,x903
0.0064 ± 0.0239,x1273
0.0064 ± 0.0210,x1008
0.0057 ± 0.0196,x1079
0.0056 ± 0.0206,x1105
0.0054 ± 0.0193,x1434


In [11]:
eli5.show_weights(classifier, vec=vectorizer, top=10)

Weight,Feature
0.0189 ± 0.0492,bad
0.0130 ± 0.0385,worst
0.0076 ± 0.0256,boring
0.0072 ± 0.0228,supposed
0.0065 ± 0.0210,nothing
0.0064 ± 0.0239,stupid
0.0064 ± 0.0210,plot
0.0057 ± 0.0196,reason
0.0056 ± 0.0206,ridiculous
0.0054 ± 0.0193,waste


In [12]:
eli5.show_weights(classifier, vec=vectorizer, top=10, target_names=['neg', 'pos'], targets=[1])

Weight,Feature
0.0189 ± 0.0492,bad
0.0130 ± 0.0385,worst
0.0076 ± 0.0256,boring
0.0072 ± 0.0228,supposed
0.0065 ± 0.0210,nothing
0.0064 ± 0.0239,stupid
0.0064 ± 0.0210,plot
0.0057 ± 0.0196,reason
0.0056 ± 0.0206,ridiculous
0.0054 ± 0.0193,waste


In [13]:
doc = documents[414]
import eli5
eli5.explain_prediction(classifier, doc, vec=vectorizer,target_names=['neg','pos'], top=20)

Explanation(estimator="RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features='auto',\n                       max_leaf_nodes=None, max_samples=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, n_estimators=1000,\n                       n_jobs=None, oob_score=False, random_state=0, verbose=0,\n                       warm_start=False)", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble (or a single tree for DecisionTreeClassifier).\nEach node of the tree has an output score, and contribution of a feature\non the decision path is how much the score changes from parent to child.\nWeights of all features sum to the output score or proba of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision path', is_regression=False, targets=[TargetExplanation(target='neg', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.4950406250000002, std=None, value=1.0), FeatureWeight(feature='dull', weight=0.05598982916381569, std=None, value=1.0), FeatureWeight(feature='stupid', weight=0.05366973758521386, std=None, value=1.0), FeatureWeight(feature='mess', weight=0.05219755534142953, std=None, value=1.0), FeatureWeight(feature='unfunny', weight=0.04050102485209005, std=None, value=1.0), FeatureWeight(feature='minutes', weight=0.027609491575814184, std=None, value=1.0), FeatureWeight(feature='even', weight=0.022325401570049214, std=None, value=2.0), FeatureWeight(feature='poor', weight=0.02113767060236639, std=None, value=1.0), FeatureWeight(feature='point', weight=0.008323142013434983, std=None, value=1.0), FeatureWeight(feature='jennifer', weight=0.007496297585323589, std=None, value=1.0), FeatureWeight(feature='credits', weight=0.006781546095395709, std=None, value=1.0), FeatureWeight(feature='wonder', weight=0.006608434068024137, std=None, value=1.0), FeatureWeight(feature='cartoon', weight=0.006557670694450483, std=None, value=1.0)], neg=[FeatureWeight(feature='life', weight=-0.01398118065874261, std=None, value=3.0), FeatureWeight(feature='bad', weight=-0.013500516104473928, std=None, value=0.0), FeatureWeight(feature='world', weight=-0.011800632292239165, std=None, value=1.0), FeatureWeight(feature='nhe', weight=-0.00816837573831147, std=None, value=1.0), FeatureWeight(feature='plot', weight=-0.0075958989831214335, std=None, value=0.0), FeatureWeight(feature='ni', weight=-0.006560491307677435, std=None, value=0.0), FeatureWeight(feature='worst', weight=-0.006536175433949148, std=None, value=0.0)], pos_remaining=859, neg_remaining=583), proba=0.811, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [14]:
eli5.show_prediction(classifier, doc, vec=vectorizer, top=20)